In [1]:
# load package
import numpy as np
import pandas as pd
from collections import Counter
import lightgbm as lgb
#from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation  import train_test_split
from sklearn import metrics
from sklearn import cross_validation
import pickle
import itertools

# set path
import os
default_path = "/Users/francislin/temp/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('all_data.pickle','rb') as f:
    df= pickle.load(f)
# 區分train test
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns
del(df)

Index(['cuisine', 'id', 'used_pepper', 'used_salt', 'used_oil', 'used_garlic',
       'used_ground', 'used_fresh', 'used_sauce', 'used_sugar',
       ...
       'used_hock', 'used_rose', 'used_ciabatta', 'used_ready', 'used_regular',
       'used_glass', 'used_safflower', 'used_jeera', 'used_Oil', 'used_oven'],
      dtype='object', length=902)

In [3]:
#dataTrain = dataTrain
#去掉y
dataTest = dataTest.drop('cuisine',axis=1)
#去掉key
X_feature = dataTrain.drop(['id','cuisine'],axis=1)
y_t =dataTrain['cuisine']

In [4]:
#把上面dataTrain.columns的結果貼到dataTrain，再去掉'Survived'
from sklearn.model_selection import train_test_split
#拿掉feature_importances低的因子
X_train, X_test, y_train, y_test = train_test_split(X_feature, y_t, test_size=0.3, random_state=13)

In [5]:
%%time
import random
from scipy.stats import expon
from scipy.stats import randint as sp_randint
# Random Search
print('Start training...')
estimator = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0) 

param_dist = {"learning_rate": expon(scale=.1),
              "feature_fraction": expon(scale=.1),
              "bagging_fraction": expon(scale=.1),
              "max_depth": [3, 7],
              #"min_samples_leaf": sp_randint(1, 11),
              "reg_lambda": expon(scale=10),
              "criterion": ["gini", "entropy"]}

n_iter_search = 20
LGBM_Rand = RandomizedSearchCV(estimator, param_distributions=param_dist,
                                   n_iter=n_iter_search, random_state=0, scoring='accuracy')
LGBM_Rand.fit(X_train, y_train)
print('Best parameters found by grid search are:', LGBM_Rand.best_params_)

Start training...
Best parameters found by grid search are: {'bagging_fraction': 0.08444277806509103, 'criterion': 'gini', 'feature_fraction': 0.10058710182988562, 'learning_rate': 0.3192946626543764, 'max_depth': 7, 'reg_lambda': 2.343015156329577}
CPU times: user 22min 50s, sys: 5min 36s, total: 28min 27s
Wall time: 11min 38s


In [9]:
# Final Model
evals_result = {} 
print('Start predicting...')
LGBM_Rand_final = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0,
                                     learning_rate = LGBM_Rand.best_params_['learning_rate'],
                              feature_fraction = LGBM_Rand.best_params_['feature_fraction'],
                                bagging_fraction = LGBM_Rand.best_params_['bagging_fraction'], 
                             #min_samples_leaf = LGBM_Rand.best_params_['min_samples_leaf'],
                             max_depth = LGBM_Rand.best_params_['max_depth'],
                                     reg_lambda = LGBM_Rand.best_params_['reg_lambda'],
                                     criterion = LGBM_Rand.best_params_['criterion']
                                    )
LGBM_Rand_final_fit = LGBM_Rand_final.fit(X_train, y_train)
print("Predicting is over")

Start predicting...
Predicting is over


In [10]:
y_pred= LGBM_Rand_final_fit.predict(X_test)

In [11]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.64285714, 0.40909091, 0.74679113, 0.80660377, 0.68949772,
       0.62483487, 0.73570325, 0.88229056, 0.49846154, 0.8438835 ,
       0.82926829, 0.76005188, 0.78389831, 0.90643425, 0.78787879,
       0.40540541, 0.73748212, 0.57079646, 0.77834525, 0.62528736])

0.7032431246800236

0.7847984580574876

In [12]:
#拿全部的dataTrain來建
LGBM_Rand_final_fit_all = LGBM_Rand_final_fit.fit(X_feature, y_t)

In [13]:
dataTest = dataTest.drop('id',axis=1)

In [14]:
LGBM_Rand_final_fit_res =LGBM_Rand_final_fit_all.predict(dataTest)

In [15]:
LGBM_Rand_final_fit_res

array(['british', 'southern_us', 'italian', ..., 'italian', 'southern_us',
       'mexican'], dtype=object)

In [16]:
submit=pd.read_csv("/Users/francislin/jrml/What's Cooking/data/sample_submission.csv")

In [17]:
submit['cuisine'] = LGBM_Rand_final_fit_res

In [18]:
%pwd
submit.to_csv("/Users/francislin/jrml/What's Cooking/submit_to_kaggle.csv", index= False)

'/Users/francislin/temp'